In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
df = pd.read_csv('/content/drive/MyDrive/6962 final project/data/Daily_Precipitation/Pakistan_merged.csv')

In [ ]:
cchf = pd.read_excel('/content/drive/MyDrive/6962 final project/CCHF - Total ProMED Archive.xlsx')

In [ ]:
cchf

,diseasename,place,country,lat,lon,alert_id,feed_id,summary,issue_date,load_date,content
0,Crimean-Congo Hemorrhagic Fever,Pakistan,Pakistan,29.967020,69.385971,2187141,1,PROMED: CCHF - Pakistan,1995-09-13 23:50:00,2008-12-30 11:02:31,"<PRE>\n\n\nDate: Wed, 13 Sep 1995 05:17:51 -07..."
1,Crimean-Congo Hemorrhagic Fever,Serbia,Serbia,43.867039,20.899031,2187683,1,PROMED: CCHF - Serbia: RFI,1996-01-09 23:50:00,2008-12-30 11:43:03,"<PRE>\n\n\n\nFrom: Jack Woodall <<a href=""mail..."
2,Crimean-Congo Hemorrhagic Fever,South Africa,South Africa,-28.998529,25.092541,2189483,1,PRO/AH/EDR> Congo hemorrhagic fever - South Af...,1996-11-05 23:50:00,2008-12-30 16:25:44,<PRE>\n\n\nCONGO HEMORRHAGIC FEVER - SOUTH AFR...
3,Crimean-Congo Hemorrhagic Fever,South Africa,South Africa,-28.998529,25.092541,2189485,1,PRO/AH/EDR> Congo hemorrhagic fever - South Af...,1996-11-05 23:50:00,2008-12-30 16:25:51,<PRE>\n\n\nCONGO HEMORRHAGIC FEVER - SOUTH AFR...
4,Crimean-Congo Hemorrhagic Fever,South Africa,South Africa,-28.998529,25.092541,2189488,1,PRO/AH/EDR> Congo hemorrhagic fever - South Af...,1996-11-05 23:50:00,2008-12-30 16:26:01,<PRE>\n\n\nCONGO HEMORRHAGIC FEVER - SOUTH AFR...
...,...,...,...,...,...,...,...,...,...,...,...
1008,Crimean-Congo Hemorrhagic Fever,Mali,Mali,17.350281,-3.524420,6958443,18,PRO/FR> Fièvre hémorragique Crimée-Congo - Mal...,2020-02-06 12:31:12,2020-02-06 12:31:12,FIÈVRE HÉMORRAGIQUE CRIMÉE-CONGO - MALI (02): ...
1009,Crimean-Congo Hemorrhagic Fever,Mali,Mali,17.350281,-3.524420,6963297,1,PRO/AH/EDR> Crimean-Congo hem. fever - Africa ...,2020-02-07 12:43:04,2020-02-07 12:43:04,CRIMEAN-CONGO HEMORRHAGIC FEVER - AFRICA (03):...
1010,Crimean-Congo Hemorrhagic Fever,"Mopti, Mopti, Mali",Mali,14.493020,-4.194200,6963297,1,PRO/AH/EDR> Crimean-Congo hem. fever - Africa ...,2020-02-07 12:43:04,2020-02-07 12:43:04,CRIMEAN-CONGO HEMORRHAGIC FEVER - AFRICA (03):...
1011,Crimean-Congo Hemorrhagic Fever,"North West Province, South Africa",South Africa,-26.500000,26.000000,6980424,1,PRO/AH/EDR> Crimean-Congo hem. fever - Africa ...,2020-02-15 11:33:27,2020-02-15 11:33:27,CRIMEAN-CONGO HEMORRHAGIC FEVER - AFRICA (04):...


In [ ]:
df.head()

,time,latitude,longitude,precip,target_label
0,05/19/2015,24,61,0.014087,0
1,05/19/2015,24,62,0.000000,0
2,05/19/2015,24,63,0.000000,0
3,05/19/2015,24,64,0.001400,0
4,05/19/2015,24,65,0.169249,0


In [ ]:
min(df['latitude']), max(df['latitude']), min(df['longitude']), max(df['longitude'])

(24, 35, 61, 75)

In [ ]:
import datetime
df['time_dt'] = pd.to_datetime(df['time'])

In [ ]:
alert_date = list(df.loc[df['target_label'] == 1]['time_dt'])

In [ ]:
lat_range = max(df['latitude']) - min(df['latitude']) + 1
lon_range = max(df['longitude']) - min(df['longitude']) + 1

lat_list =  (max(df['longitude']) - min(df['longitude']) + 1) * list(range(min(df['latitude']), max(df['latitude'])+1))
lat_list.sort()
lon_list = (max(df['latitude']) - min(df['latitude']) + 1) * list(range(min(df['longitude']), max(df['longitude'])+1))

lat_list = lat_list * len(set(alert_date))
lon_list = lon_list * len(set(alert_date))

time_list = list(set(alert_date)) * lat_range * lon_range
time_list.sort()

In [ ]:
assert(len(lat_list) == len(lon_list) and len(lon_list) == len(time_list))

In [ ]:
df = df.sort_values(by=['latitude','longitude','time_dt'])
df.reset_index(inplace = True, drop= True)
alerts = list(df.loc[df['target_label'] == 1].index)

In [ ]:
import datetime
from datetime import timedelta

In [ ]:
import random
df_output = pd.DataFrame(columns=['date','latitude','longitude','label','p_14','p_13','p_12','p_11',
                                  'p_10','p_9','p_8','p_7','p_6','p_5','p_4','p_3','p_2','p_1','p_0'])
index_count = 0

for i in alerts:
  #Sampling label 0 
  _lon = random.sample(lon_list, 1)
  _lat = random.sample(lat_list, 1)
  a = df.loc[df['time'] == df.loc[i,'time']]
  b = a.loc[a['latitude'] == _lat[0]]
  c = b.loc[b['longitude'] == _lon[0]]

  #Adding label 0 data into df
  _data = dict()
  _data['date'] = df.loc[c.index[0],'time']
  _data['latitude'] = df.loc[c.index[0],'latitude']
  _data['longitude'] = df.loc[c.index[0],'longitude']
  _data['label'] = df.loc[c.index[0],'target_label']
  for j in list(range(0,15)):
            _data['p_' + str(j)] = df.loc[c.index[0] - j,'precip']
            # df.loc[c.index - j, 'precip']
  df_output.loc[index_count] = _data
  index_count += 1


  #Collecting label 1 data
  _data = dict()
  _data['date'] = df.loc[i,'time']
  _data['latitude'] = df.loc[i,'latitude']
  _data['longitude'] = df.loc[i,'longitude']
  _data['label'] = df.loc[i,'target_label']
  for j in list(range(0,15)):
            _data['p_' + str(j)] = df.loc[i - j, 'precip']
  df_output.loc[index_count] = _data
  index_count += 1


In [ ]:
df_output

,date,latitude,longitude,label,p_14,p_13,p_12,p_11,p_10,p_9,p_8,p_7,p_6,p_5,p_4,p_3,p_2,p_1,p_0
0,09/13/2014,33,70,0,0.249097,2.054005,0.966753,5.404934,3.886570,3.472538,0.477327,1.051291,0.540116,1.325990,1.416625,0.001061,0.000000,0.000000,0.000000
1,09/13/2014,24,67,1,0.110025,0.091635,0.224904,0.611018,0.220788,0.183348,0.081854,1.155431,0.324847,0.000099,0.000334,0.000000,0.000000,0.000000,0.000000
2,10/01/2014,31,70,0,0.072972,0.465228,0.025869,0.221584,0.198461,0.097829,1.813652,0.415277,1.433205,2.623857,0.017444,0.677476,0.321304,0.284268,0.070771
3,10/01/2014,24,67,1,0.003662,0.002320,0.009753,0.009852,0.047320,0.131508,4.934797,12.078712,2.038262,0.254640,1.093849,0.004421,0.037115,0.002950,0.000000
4,10/21/2014,25,67,0,0.045791,0.054659,0.008469,0.000000,0.022700,0.002371,0.005569,0.000000,0.043127,0.126326,0.057107,0.063286,0.155787,0.333094,0.047983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,11/07/2015,34,72,1,0.042128,0.024238,0.012106,0.019312,0.196594,0.011638,0.018238,0.150627,0.098120,0.479814,0.128470,0.023314,0.014011,0.001798,0.020321
210,09/27/2016,24,69,0,0.952887,0.204477,0.035986,0.153831,0.058550,0.054948,0.134134,0.133361,0.060795,0.000000,0.001306,0.005916,0.000000,0.053912,0.028301
211,09/27/2016,34,72,1,0.003526,0.002105,0.024935,0.220844,7.041093,9.251787,0.437008,0.000706,0.205277,0.047039,0.169389,0.127242,0.029989,0.188984,0.502225
212,09/27/2009,31,70,0,1.222426,1.341620,0.271251,0.231076,0.083956,0.260740,2.482223,0.854684,0.051626,0.004279,0.040570,0.679182,0.080143,0.119106,0.162286


In [ ]:
# _precip_dict = {(df.at[0,'time_dt'],df.at[0,'longitude'],df.at[0,'latitude']) : (df.at[0,'precip'],df.at[0,'target_label']) }
# _last_time = df.at[0,'time_dt']
# _first_time = df.at[0,'time_dt']

# for i in range(1, len(df)):
#   _time = df.at[i,'time_dt']
#   _lon = df.at[i,'longitude']
#   _lat = df.at[i,'latitude']
#   _precip = df.at[i,'precip']
#   _label = df.at[i,'target_label']


#   if (_last_time - _time).days <= 1:
#     _precip_dict[(df.at[i,'time_dt'],df.at[i,'longitude'],df.at[i,'latitude'])] = (df.at[i,'precip'],df.at[i,'target_label'])
#     _last_time = _time
#   else:
#     _lat_list =  (max(df['longitude']) - min(df['longitude']) + 1) * list(range(min(df['latitude']), max(df['latitude'])+1))
#     _lat_list.sort()
#     _lon_list = (max(df['latitude']) - min(df['latitude']) + 1) * list(range(min(df['longitude']), max(df['longitude'])+1))
#     _time_list = [_time] * len(_lon_list)#where label = 1

#     df_loc = pd.DataFrame({'date':time_list,'latitude':lat_list, 'longitude':lon_list,'label': [0]*len(lon_list)},
#                           columns=['date','latitude','longitude','label','p_14','p_13','p_12','p_11','p_10','p_9','p_8','p_7','p_6','p_5','p_4','p_3','p_2','p_1','p_0'])
    
#     counter = 0
#     for d in list(range(0,(_last_time - _first_time).days+1):
#       for x in range(min(df['latitude']), max(df['latitude'])+1):
#         for y in range(min(df['longitude']), max(df['longitude'])+1):
#           # if d == _time and x == _lat and y == _lon:
#           #   df_loc.loc[counter,'label'] = 1
#           for z in list(range(0,15)):
#             df_loc.loc[counter,'p_' + str(z)] = _precip_dict[(_time - days(z), x, y][0]
#             df_loc.loc[counter,'label'] = _precip_dict[(_time - days(z), x, y][1]
#           counter += 1

#     _precip_dict = {(df.at[i,'time_dt'],df.at[i,'longitude'],df.at[i,'latitude']) : (df.at[i,'precip'],df.at[i,'target_label']) }
#     _first_time = _time
#   # for lat in range(min(df['latitude']), max(df['latitude'])+1):
#   #   for lon in range(min(df['longitude']), max(df['longitude'])+1):

#Create 7_days_avg, 14_days_avg


In [ ]:
'p_14','p_13','p_12','p_11','p_10','p_9','p_8','p_7','p_6','p_5','p_4','p_3','p_2','p_1'

In [ ]:
sum_7 = df_output.copy()['p_0']
for i in ['p_6','p_5','p_4','p_3','p_2','p_1']:
  sum_7 += df_output[i]
df_output['avg_7'] = sum_7/7

sum_14 = df_output.copy()['p_0']
for i in ['p_13','p_12','p_11','p_10','p_9','p_8','p_7','p_6','p_5','p_4','p_3','p_2','p_1']:
  sum_14 += df_output[i]
df_output['avg_14'] = sum_14/14

In [ ]:
df_output

,date,latitude,longitude,label,p_14,p_13,p_12,p_11,p_10,p_9,p_8,p_7,p_6,p_5,p_4,p_3,p_2,p_1,p_0,avg_7,avg_14
0,09/13/2014,33,70,0,0.249097,2.054005,0.966753,5.404934,3.886570,3.472538,0.477327,1.051291,0.540116,1.325990,1.416625,0.001061,0.000000,0.000000,0.000000,0.469113,1.471229
1,09/13/2014,24,67,1,0.110025,0.091635,0.224904,0.611018,0.220788,0.183348,0.081854,1.155431,0.324847,0.000099,0.000334,0.000000,0.000000,0.000000,0.000000,0.046469,0.206733
2,10/01/2014,31,70,0,0.072972,0.465228,0.025869,0.221584,0.198461,0.097829,1.813652,0.415277,1.433205,2.623857,0.017444,0.677476,0.321304,0.284268,0.070771,0.775475,0.619016
3,10/01/2014,24,67,1,0.003662,0.002320,0.009753,0.009852,0.047320,0.131508,4.934797,12.078712,2.038262,0.254640,1.093849,0.004421,0.037115,0.002950,0.000000,0.490177,1.474678
4,10/21/2014,25,67,0,0.045791,0.054659,0.008469,0.000000,0.022700,0.002371,0.005569,0.000000,0.043127,0.126326,0.057107,0.063286,0.155787,0.333094,0.047983,0.118101,0.065748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,11/07/2015,34,72,1,0.042128,0.024238,0.012106,0.019312,0.196594,0.011638,0.018238,0.150627,0.098120,0.479814,0.128470,0.023314,0.014011,0.001798,0.020321,0.109407,0.085614
210,09/27/2016,24,69,0,0.952887,0.204477,0.035986,0.153831,0.058550,0.054948,0.134134,0.133361,0.060795,0.000000,0.001306,0.005916,0.000000,0.053912,0.028301,0.021461,0.066108
211,09/27/2016,34,72,1,0.003526,0.002105,0.024935,0.220844,7.041093,9.251787,0.437008,0.000706,0.205277,0.047039,0.169389,0.127242,0.029989,0.188984,0.502225,0.181449,1.303473
212,09/27/2009,31,70,0,1.222426,1.341620,0.271251,0.231076,0.083956,0.260740,2.482223,0.854684,0.051626,0.004279,0.040570,0.679182,0.080143,0.119106,0.162286,0.162456,0.475910


In [ ]:
%cd '/content/drive/MyDrive/6962 final project/data/'

/content/drive/.shortcut-targets-by-id/12pjah520zwJMQos4SHRxl7lFoUlEuU7h/6962 final project/data


In [ ]:
df_output.to_csv('Pakistan_with_Features.csv')